Erstellen einer FHIR Patient Ressource mit dem Python fhirclient

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

In [9]:
from IPython.display import IFrame
import fhirclient
import fhirclient.models.patient as p

In [15]:
# Patient Ressource in der FHIR Spezifikation.
IFrame('http://hl7.org/fhir/patient.html#resource', width=1200, height=330)

In [6]:
# Instanziieren eines Patienten
patient = p.Patient()

In [11]:
# help() Funktion, um mehr Informationen zu erhalten
help(patient)

Help on Patient in module fhirclient.models.patient object:

class Patient(fhirclient.models.domainresource.DomainResource)
 |  Information about an individual or animal receiving health care services.
 |  
 |  Demographics and other administrative information about an individual or
 |  animal receiving care or other health-related services.
 |  
 |  Method resolution order:
 |      Patient
 |      fhirclient.models.domainresource.DomainResource
 |      fhirclient.models.resource.Resource
 |      fhirclient.models.fhirabstractresource.FHIRAbstractResource
 |      fhirclient.models.fhirabstractbase.FHIRAbstractBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jsondict=None, strict=True)
 |      Initialize all valid properties.
 |      
 |      :raises: FHIRValidationError on validation errors, unless strict is False
 |      :param dict jsondict: A JSON dictionary to use for initialization
 |      :param bool strict: If True (the default), invalid variab

In [8]:
for property in patient.elementProperties():
    print(property)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('active', 'active', <class 'bool'>, False, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('birthDate', 'birthDate', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, None, False)
('communication', 'communication', <class 'fhirclient.models.patient.PatientCommunication'>, True, None, False)
('contact',

Name des Patienten / dr Patientin

In [14]:
IFrame('http://hl7.org/fhir/datatypes.html#HumanName', width=1200, height=330)

In [12]:
import fhirclient.models.humanname as hn

In [20]:
name = hn.HumanName()
name.family = 'Bach'
name.given = ['Johann', 'Sebastian']
name.text = 'Johann Sebastian Bach'
name.use = 'official'
patient.name = [name]

Geburtsdatum

In [17]:
IFrame('http://hl7.org/fhir/datatypes.html#date', width=1200, height=330)

In [13]:
import fhirclient.models.fhirdate as fd

In [16]:
dateOfBirth = fd.FHIRDate('1967-05-25')
patient.birthDate = dateOfBirth

In [27]:
import json
from json import dumps

In [21]:
print(json.dumps(patient.as_json(), indent=4))

{
    "birthDate": "1967-05-25",
    "name": [
        {
            "family": "Bach",
            "given": [
                "Johann",
                "Sebastian"
            ],
            "text": "Johann Sebastian Bach",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}


In [28]:
import requests
from requests import get, post, put

In [44]:
fhir_test_server = 'http://localhost:4080'
#fhir_test_server = 'https://vonk.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [45]:
response = requests.post(f'{fhir_test_server}/Patient/$validate', headers = headers, data = dumps(patient.as_json()))
data = response.json()


In [46]:
print(response.status_code)

200


In [47]:
print(json.dumps(data, indent=4))

{
    "resourceType": "OperationOutcome",
    "id": "96a3d0dd-e0ba-42ee-a248-85d48fa9745a",
    "meta": {
        "versionId": "42e59302-e1a9-4755-a9f8-4651aaf3351b",
        "lastUpdated": "2020-11-23T09:05:16.495+00:00"
    },
    "issue": [
        {
            "severity": "information",
            "code": "informational",
            "diagnostics": "The operation was successful"
        }
    ]
}


In [48]:
response = requests.post(f'{fhir_test_server}/Patient', headers = headers, data = dumps(patient.as_json()))
data = response.json()

In [49]:
print(response.status_code)

201


In [51]:
print(json.dumps(data, indent=4))

{
    "resourceType": "Patient",
    "birthDate": "1967-05-25",
    "name": [
        {
            "family": "Bach",
            "given": [
                "Johann",
                "Sebastian"
            ],
            "text": "Johann Sebastian Bach",
            "use": "official"
        }
    ],
    "id": "71499131-972c-4785-a3e8-4e16dc42e191",
    "meta": {
        "versionId": "637a3f6b-6508-42f5-9cec-4c40600319e8",
        "lastUpdated": "2020-11-23T09:06:40.969+00:00"
    }
}
